# Gemma 2 Evaluation - 02 -

In diesem notebook wird untersucht wie gut Gemma 2 die gewünschten Entitäten (EVENT, TOPIC, DATE, TIME, LOC) erkennt.

Es wird folgendes Model genutzt: **"gemma-2-2b-it"**   
kleinstes Modell der Gemma2 Modelle  
(2B model was trained with 2 trillion tokens)

Die Performance von Gemma 2 wird auf dem Ground Truth untersucht.

**erste Quantitative Analyse - Berechnung von Precision, Recall und F1-Score - nur als erster Vergleich zw den unterschiedlichen Prompts**

--> zwei Durchläufe


--> Bei den ersten Ausgaben ([siehe notebook 01_gemma2_prompt_test1](01_gemma2_prompt_test1.ipynb)) zeigte sich , dass nicht für jeden Text Entitäten extrahiert wurden.   
Es wurden Fehler angezeigt, die die Ausgabe als Json betreffen. Zudem wurden Wörter "erfunden", die nicht im Text vorkommen. Das Modell halluzinierte also.   
Der Prompt wird dementsprechend etwas modifiziert. 


----

**Prompt:**

prompt = f"""
        Der folgende Text stammt von einem Veranstaltungsplakat. Extrahiere, wenn möglich, die folgenden Informationen:
        
        - **Ort** (z. B. Straßen mit Hausnummern, Plätze, Gebäude, Stadtteile, Städte)
        - **Datum** (z. B. "21.12.", "Dienstag", "9. November", "8.8.2021")
        - **Zeit** (z. B. "19 Uhr", "18:00 Uhr")
        - **Veranstaltungsart** (z. B. "Kundgebung", "Demonstration", "Feier")
        - **Thema** (z. B. "Solidarität mit...", "Gedenken an...", "für Frieden")
        
        Regeln:
        - Gib nur Informationen an, bei denen du dir sicher bist.
        - Verwende **ausschließlich Wörter aus dem Originaltext**.
        - Nicht alle Felder müssen vorhanden sein.
        - Mehrfache Einträge pro Feld sind möglich.
        
        Gib das Ergebnis exakt im folgenden Format zurück:
        
        ```json
        {{
          "Ort": [...],
          "Datum": [...],
          "Zeit": [...],
          "Kategorie": [...],
          "Thema": [...]
        }}

        
        Hier ist der Text:
        {text}
        """

**---> es wurden zwei Durchläufe mit demselben Prompt gemacht**

---

In [1]:
from huggingface_hub import login
import os

login(token=os.getenv("HUGGINGFACE_TOKEN"))


In [16]:
!pip install transformers torch accelerate

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
model_name = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="cpu") #habe kein gpu, deshab cpu
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="cpu")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

---

### Auf Datensatz iterieren und Output im Json-Format speichern

In [11]:
# Funktion extract_entities
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


# Model und Pipeline
model_name = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=-1)

def extract_entities(data):
    results = []

    for eintrag in data:
        file_name = eintrag["file_name"]
        text = eintrag["text"]
    
        prompt = f"""
        Der folgende Text stammt von einem Veranstaltungsplakat. Extrahiere, wenn möglich, die folgenden Informationen:
        
        - **Ort** (z. B. Straßen mit Hausnummern, Plätze, Gebäude, Stadtteile, Städte)
        - **Datum** (z. B. "21.12.", "Dienstag", "9. November", "8.8.2021")
        - **Zeit** (z. B. "19 Uhr", "18:00 Uhr")
        - **Veranstaltungsart** (z. B. "Kundgebung", "Demonstration", "Feier")
        - **Thema** (z. B. "Solidarität mit...", "Gedenken an...", "für Frieden")
        
        Regeln:
        - Gib nur Informationen an, bei denen du dir sicher bist.
        - Verwende **ausschließlich Wörter aus dem Originaltext**.
        - Nicht alle Felder müssen vorhanden sein.
        - Mehrfache Einträge pro Feld sind möglich.
        
        Gib das Ergebnis exakt im folgenden Format zurück:
        
        ```json
        {{
          "Ort": [...],
          "Datum": [...],
          "Zeit": [...],
          "Kategorie": [...],
          "Thema": [...]
        }}

        
        Hier ist der Text:
        {text}
        """
        
        try:
            output = generator(prompt, max_new_tokens=400)[0]["generated_text"]
            extracted_json = output.split("```json")[-1].split("```")[0].strip()  # nur Output nach "```json" wird genommen
            extracted = json.loads(extracted_json)
        
            results.append({
                "file_name": file_name,
                "entities": extracted
            })
        
        except Exception as e:
            print(f"Fehler bei {file_name}: {e}")
            results.append({
                "file_name": file_name,
                "entities": None,
                "error": str(e)
            })
        
    return results




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


In [12]:
import sys
import os
import json
#sys.path.append(os.path.abspath(".."))
#from ressourcen_monitor import monitor

with open("../../data/data_annotated.json", encoding="utf-8") as f:
    data = json.load(f)

#@monitor(full_name="Gemma2 Prompt1_2 NER")
#def run_ner():
#    return extract_entities(data)
#results = run_ner()

results = extract_entities(data)

with open("../../data/NER/gemma2/prompt2/gemma2_entity_output_prompt2_2.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)


Fehler bei 0015.jpg: Expecting value: line 2 column 19 (char 20)
Fehler bei 0023.jpg: Expecting value: line 2 column 19 (char 20)
Fehler bei 0027.jpg: Expecting value: line 2 column 19 (char 20)
Fehler bei 0039.jpeg: Expecting ',' delimiter: line 7 column 1 (char 137)
Fehler bei 0057.jpg: Invalid control character at: line 2 column 321 (char 322)
Fehler bei 0064.jpg: Expecting value: line 2 column 19 (char 20)
Fehler bei 0080.jpg: Expecting value: line 2 column 19 (char 20)
Fehler bei 0089.jpeg: Expecting value: line 2 column 19 (char 20)
Fehler bei 0091.jpg: Expecting ',' delimiter: line 7 column 1 (char 160)
Fehler bei 0093.jpg: Expecting ',' delimiter: line 5 column 394 (char 546)
Fehler bei 0100.jpg: Expecting value: line 2 column 19 (char 20)
Fehler bei 0101.jpg: Expecting value: line 2 column 19 (char 20)
Fehler bei 0116.jpg: Expecting value: line 2 column 19 (char 20)
Fehler bei 0122.jpg: Expecting value: line 2 column 19 (char 20)
Fehler bei 0134.jpg: Expecting value: line 2 co

### Json-Datei bereinigen

In [13]:
# auf "leere"/ ungültige Einträge prüfen
import json

# JSON laden
with open("../../data/NER/gemma2/prompt2/gemma2_entity_output_prompt2_2.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Zähler initialisieren
gesamt = len(data)
ungueltig = 0
mit_error = 0
gueltig = 0

for eintrag in data:
    entities = eintrag.get("entities")
    
    if not entities or not isinstance(entities, dict):
        ungueltig += 1
        if "error" in eintrag:
            mit_error += 1
    else:
        gueltig += 1

# Ausgabe
print(f"Gesamtanzahl Dateien: {gesamt}")
print(f"Ungültige Einträge (keine Entities oder kaputt): {ungueltig}")
print(f"  └── davon mit 'error'-Feld: {mit_error}")
print(f"Gültige Einträge (Entities vorhanden): {gueltig}")


Gesamtanzahl Dateien: 200
Ungültige Einträge (keine Entities oder kaputt): 27
  └── davon mit 'error'-Feld: 27
Gültige Einträge (Entities vorhanden): 173


In [16]:
# Datei bereingigen & Labels mappen
import json

# Label-Mapping
label_map = {
    "Ort": "LOC",
    "Datum": "DATE",
    "Zeit": "TIME",
    "Kategorie": "EVENT",
    "Thema": "TOPIC"
}

# JSON laden
with open("../../data/NER/gemma2/prompt2/gemma2_entity_output_prompt2_2.json", "r", encoding="utf-8") as f:
    data = json.load(f)

bereinigt = []
for eintrag in data:
    entities = eintrag.get("entities")

    if not entities or not isinstance(entities, dict):
        continue  # überspringen, wenn kaputt oder leer
    
    # null-Werte rausfiltern und Keys mappen
    entities_cleaned = {
        label_map.get(k, k): v
        for k, v in entities.items()
        if v is not None
    }

    # überspringen, wenn nach dem Filtern nichts übrig ist (optional)
    if not entities_cleaned:
        continue

    eintrag["entities"] = entities_cleaned
    bereinigt.append(eintrag)

# Speichern
with open("../../data/NER/gemma2/prompt2/gemma2_entity_output_prompt2_2_cleaned.json", "w", encoding="utf-8") as f:
    json.dump(bereinigt, f, ensure_ascii=False, indent=2)


In [17]:
# Json in Struktur des Goldstandard umwandeln

import json

# Datei laden
with open("../../data/NER/gemma2/prompt2/gemma2_entity_output_prompt2_2_cleaned.json", encoding="utf-8") as f:
    gemma_data = json.load(f)

# Normalisieren
def normalize_to_list(value):
    if isinstance(value, list):
        return [v.strip() for v in value if isinstance(v, str) and v.strip()]
    elif isinstance(value, str) and value.strip():
        return [value.strip()]
    else:
        return []

# In Goldstandard-ähnliche Struktur bringen
converted = []

for entry in gemma_data:
    file_name = entry.get("file_name")
    raw_entities = entry.get("entities", {})
    
    entities = []
    for label, value in raw_entities.items():
        values = normalize_to_list(value)
        for v in values:
            entities.append({
                "label": label,
                "text": v
            })
    
    converted.append({
        "file_name": file_name,
        "entities": entities
    })

# Speichern
with open("../../data/NER/gemma2_prompt2_2_as_goldstructure.json", "w", encoding="utf-8") as f:
    json.dump(converted, f, ensure_ascii=False, indent=2)



### Evaluation - 1. Durchlauf

In [7]:
# auf "leere"/ ungültige Einträge prüfen
import json

# JSON laden
with open("../../data/NER/gemma2/prompt2/gemma2_entity_output_prompt2_1_cleaned.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Zähler initialisieren
gesamt = len(data)
ungueltig = 0
mit_error = 0
gueltig = 0

for eintrag in data:
    entities = eintrag.get("entities")
    
    if not entities or not isinstance(entities, dict):
        ungueltig += 1
        if "error" in eintrag:
            mit_error += 1
    else:
        gueltig += 1

# Ausgabe
print(f"Gesamtanzahl Dateien: {gesamt}")
print(f"Ungültige Einträge (keine Entities oder kaputt): {ungueltig}")
print(f"  └── davon mit 'error'-Feld: {mit_error}")
print(f"Gültige Einträge (Entities vorhanden): {gueltig}")


Gesamtanzahl Dateien: 183
Ungültige Einträge (keine Entities oder kaputt): 0
  └── davon mit 'error'-Feld: 0
Gültige Einträge (Entities vorhanden): 183


---> bei diesem Prompt weniger fehlerhafte Dateien als im notebook 01_gemma2_evaluation

In [8]:
import json
from tqdm import tqdm
from collections import defaultdict
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd

# === Dateien laden ===
with open("../../data/data_annotated.json", encoding="utf-8") as f:
    gold_data = json.load(f)

with open("../../data/NER/gemma2/prompt2/gemma2_prompt2_1_as_goldstructure.json", encoding="utf-8") as f:
    gemma_data = json.load(f)

# === Zu vergleichende Labels ===
label_names = ["LOC", "DATE", "TIME", "EVENT", "TOPIC"]

# === Entitäten extrahieren ===
def extract_entities(entry):
    ent_dict = {}
    for ent in entry.get("entities", []):
        label = ent.get("label")
        text = ent.get("text", "").strip()
        if label and text:
            ent_dict.setdefault(label, []).append(text)
    return ent_dict

# === Mapping der Vorhersagen für schnellen Zugriff ===
gemma_dict = {entry["file_name"]: entry for entry in gemma_data}

# === Vergleich vorbereiten ===
y_true_all = []
y_pred_all = []
y_true_per_label = defaultdict(list)
y_pred_per_label = defaultdict(list)

# === Vergleichsloop über alle Goldstandard-Dateien ===
for gold_entry in tqdm(gold_data):
    file_name = gold_entry["file_name"]
    gold_ents = extract_entities(gold_entry)
    pred_entry = gemma_dict.get(file_name)
    pred_ents = extract_entities(pred_entry) if pred_entry else {}

    for label in label_names:
        gold_texts = set(t.strip() for t in gold_ents.get(label, []) if isinstance(t, str))
        val = pred_ents.get(label)

        if isinstance(val, list):
            pred_texts = set(t.strip() for t in val if isinstance(t, str))
        elif isinstance(val, str):
            pred_texts = {val.strip()}
        else:
            pred_texts = set()

        for text in gold_texts:
            if text in pred_texts:
                y_true_all.append(1)
                y_pred_all.append(1)
                y_true_per_label[label].append(1)
                y_pred_per_label[label].append(1)
                pred_texts.remove(text)
            else:
                y_true_all.append(1)
                y_pred_all.append(0)
                y_true_per_label[label].append(1)
                y_pred_per_label[label].append(0)

        for text in pred_texts:
            y_true_all.append(0)
            y_pred_all.append(1)
            y_true_per_label[label].append(0)
            y_pred_per_label[label].append(1)

# === Gesamtmetriken ===
precision, recall, f1, _ = precision_recall_fscore_support(
    y_true_all, y_pred_all, average="binary", zero_division=0
)

print("\n🔍 Gesamtergebnis (alle Goldstandard-Dateien)")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")
print(f"F1-Score:  {f1:.3f}")

# === Pro-Label-Metriken ===
rows = []
for label in label_names:
    y_true = y_true_per_label[label]
    y_pred = y_pred_per_label[label]

    if not y_true and not y_pred:
        precision = recall = f1 = 0.0
    else:
        precision, recall, f1, _ = precision_recall_fscore_support(
            y_true, y_pred, average="binary", zero_division=0
        )

    rows.append({
        "Label": label,
        "Precision": round(precision, 3),
        "Recall": round(recall, 3),
        "F1-Score": round(f1, 3),
        "Anzahl in Gold": sum(y_true),
        "Anzahl Predicted": sum(y_pred)
    })

# Ausgabe
df_metrics = pd.DataFrame(rows).sort_values("Label")
print("\n📊 Metriken pro Kategorie:")
print(df_metrics.to_string(index=False))


100%|██████████████████████████████████████| 200/200 [00:00<00:00, 86355.86it/s]


🔍 Gesamtergebnis (alle Goldstandard-Dateien)
Precision: 0.288
Recall:    0.185
F1-Score:  0.225

📊 Metriken pro Kategorie:
Label  Precision  Recall  F1-Score  Anzahl in Gold  Anzahl Predicted
 DATE      0.490   0.328     0.393             290               194
EVENT      0.158   0.110     0.129             237               165
  LOC      0.266   0.117     0.162             403               177
 TIME      0.445   0.380     0.410             213               182
TOPIC      0.055   0.039     0.046             255               182


#### nur Dateien mit erkannten Entitäten in die Auswertung mit einbeziehen

In [9]:
import json
from tqdm import tqdm
from collections import defaultdict
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd

# === Dateien laden ===
with open("../../data/data_annotated.json", encoding="utf-8") as f:
    gold_data = json.load(f)

with open("../../data/NER/gemma2/prompt2/gemma2_prompt2_1_as_goldstructure.json", encoding="utf-8") as f:
    gemma_data = json.load(f)

# === Zu vergleichende Labels ===
label_names = ["LOC", "DATE", "TIME", "EVENT", "TOPIC"]

# === Entitäten extrahieren ===
def extract_entities(entry):
    ent_dict = {}
    for ent in entry.get("entities", []):
        label = ent.get("label")
        text = ent.get("text", "").strip()
        if label and text:
            ent_dict.setdefault(label, []).append(text)
    return ent_dict

# === Gemeinsame Dateien filtern ===
gold_files = {entry["file_name"] for entry in gold_data}
gemma_files = {entry["file_name"] for entry in gemma_data}
common_files = gold_files & gemma_files

filtered_gold = [entry for entry in gold_data if entry["file_name"] in common_files]
gemma_dict = {entry["file_name"]: entry for entry in gemma_data if entry["file_name"] in common_files}

# === Vergleich vorbereiten ===
y_true_all = []
y_pred_all = []

# pro Label getrennte Zählung
y_true_per_label = defaultdict(list)
y_pred_per_label = defaultdict(list)

# === Vergleichsloop ===
for gold_entry in tqdm(filtered_gold):
    file_name = gold_entry["file_name"]
    gold_ents = extract_entities(gold_entry)
    pred_entry = gemma_dict.get(file_name)
    pred_ents = extract_entities(pred_entry) if pred_entry else {}

    for label in label_names:
        gold_texts = set(t.strip() for t in gold_ents.get(label, []) if isinstance(t, str))
        val = pred_ents.get(label)

        if isinstance(val, list):
            pred_texts = set(t.strip() for t in val if isinstance(t, str))
        elif isinstance(val, str):
            pred_texts = {val.strip()}
        else:
            pred_texts = set()

        # True Positives und False Negatives
        for text in gold_texts:
            if text in pred_texts:
                y_true_all.append(1)
                y_pred_all.append(1)
                y_true_per_label[label].append(1)
                y_pred_per_label[label].append(1)
                pred_texts.remove(text)
            else:
                y_true_all.append(1)
                y_pred_all.append(0)
                y_true_per_label[label].append(1)
                y_pred_per_label[label].append(0)

        # Verbleibende Vorhersagen = False Positives
        for text in pred_texts:
            y_true_all.append(0)
            y_pred_all.append(1)
            y_true_per_label[label].append(0)
            y_pred_per_label[label].append(1)

# === Gesamtmetriken ===
precision, recall, f1, _ = precision_recall_fscore_support(
    y_true_all, y_pred_all, average="binary", zero_division=0
)

print("\n🔍 Gesamtergebnis (nur gemeinsame Dateien)")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")
print(f"F1-Score:  {f1:.3f}")

# === Metriken pro Label ===
rows = []
for label in label_names:
    y_true = y_true_per_label[label]
    y_pred = y_pred_per_label[label]

    if not y_true and not y_pred:
        precision = recall = f1 = 0.0
    else:
        precision, recall, f1, _ = precision_recall_fscore_support(
            y_true, y_pred, average="binary", zero_division=0
        )

    rows.append({
        "Label": label,
        "Precision": round(precision, 3),
        "Recall": round(recall, 3),
        "F1-Score": round(f1, 3),
        "Anzahl in Gold": sum(y_true),
        "Anzahl Predicted": sum(y_pred)
    })

# Ausgabe als Tabelle
df_metrics = pd.DataFrame(rows).sort_values("Label")
print("\n📊 Metriken pro Kategorie:")
print(df_metrics.to_string(index=False))


100%|██████████████████████████████████████| 183/183 [00:00<00:00, 80753.04it/s]


🔍 Gesamtergebnis (nur gemeinsame Dateien)
Precision: 0.288
Recall:    0.204
F1-Score:  0.239

📊 Metriken pro Kategorie:
Label  Precision  Recall  F1-Score  Anzahl in Gold  Anzahl Predicted
 DATE      0.490   0.365     0.419             260               194
EVENT      0.158   0.124     0.139             209               165
  LOC      0.266   0.131     0.175             360               177
 TIME      0.445   0.418     0.431             194               182
TOPIC      0.055   0.041     0.047             245               182


In [ ]:
Ergebnis nur wenig besser

--- 
### Evaluation - 2. Durchlauf

In [18]:
# auf "leere"/ ungültige Einträge prüfen
import json

# JSON laden
with open("../../data/NER/gemma2/prompt2/gemma2_entity_output_prompt2_2_cleaned.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Zähler initialisieren
gesamt = len(data)
ungueltig = 0
mit_error = 0
gueltig = 0

for eintrag in data:
    entities = eintrag.get("entities")
    
    if not entities or not isinstance(entities, dict):
        ungueltig += 1
        if "error" in eintrag:
            mit_error += 1
    else:
        gueltig += 1

# Ausgabe
print(f"Gesamtanzahl Dateien: {gesamt}")
print(f"Ungültige Einträge (keine Entities oder kaputt): {ungueltig}")
print(f"  └── davon mit 'error'-Feld: {mit_error}")
print(f"Gültige Einträge (Entities vorhanden): {gueltig}")

Gesamtanzahl Dateien: 173
Ungültige Einträge (keine Entities oder kaputt): 0
  └── davon mit 'error'-Feld: 0
Gültige Einträge (Entities vorhanden): 173


In [19]:
import json
from tqdm import tqdm
from collections import defaultdict
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd

# === Dateien laden ===
with open("../../data/data_annotated.json", encoding="utf-8") as f:
    gold_data = json.load(f)

with open("../../data/NER/gemma2_prompt2_2_as_goldstructure.json", encoding="utf-8") as f:
    gemma_data = json.load(f)

# === Zu vergleichende Labels ===
label_names = ["LOC", "DATE", "TIME", "EVENT", "TOPIC"]

# === Entitäten extrahieren ===
def extract_entities(entry):
    ent_dict = {}
    for ent in entry.get("entities", []):
        label = ent.get("label")
        text = ent.get("text", "").strip()
        if label and text:
            ent_dict.setdefault(label, []).append(text)
    return ent_dict

# === Mapping der Vorhersagen für schnellen Zugriff ===
gemma_dict = {entry["file_name"]: entry for entry in gemma_data}

# === Vergleich vorbereiten ===
y_true_all = []
y_pred_all = []
y_true_per_label = defaultdict(list)
y_pred_per_label = defaultdict(list)

# === Vergleichsloop über alle Goldstandard-Dateien ===
for gold_entry in tqdm(gold_data):
    file_name = gold_entry["file_name"]
    gold_ents = extract_entities(gold_entry)
    pred_entry = gemma_dict.get(file_name)
    pred_ents = extract_entities(pred_entry) if pred_entry else {}

    for label in label_names:
        gold_texts = set(t.strip() for t in gold_ents.get(label, []) if isinstance(t, str))
        val = pred_ents.get(label)

        if isinstance(val, list):
            pred_texts = set(t.strip() for t in val if isinstance(t, str))
        elif isinstance(val, str):
            pred_texts = {val.strip()}
        else:
            pred_texts = set()

        for text in gold_texts:
            if text in pred_texts:
                y_true_all.append(1)
                y_pred_all.append(1)
                y_true_per_label[label].append(1)
                y_pred_per_label[label].append(1)
                pred_texts.remove(text)
            else:
                y_true_all.append(1)
                y_pred_all.append(0)
                y_true_per_label[label].append(1)
                y_pred_per_label[label].append(0)

        for text in pred_texts:
            y_true_all.append(0)
            y_pred_all.append(1)
            y_true_per_label[label].append(0)
            y_pred_per_label[label].append(1)

# === Gesamtmetriken ===
precision, recall, f1, _ = precision_recall_fscore_support(
    y_true_all, y_pred_all, average="binary", zero_division=0
)

print("\n🔍 Gesamtergebnis 2. Durchlauf (alle Goldstandard-Dateien)")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")
print(f"F1-Score:  {f1:.3f}")

# === Pro-Label-Metriken ===
rows = []
for label in label_names:
    y_true = y_true_per_label[label]
    y_pred = y_pred_per_label[label]

    if not y_true and not y_pred:
        precision = recall = f1 = 0.0
    else:
        precision, recall, f1, _ = precision_recall_fscore_support(
            y_true, y_pred, average="binary", zero_division=0
        )

    rows.append({
        "Label": label,
        "Precision": round(precision, 3),
        "Recall": round(recall, 3),
        "F1-Score": round(f1, 3),
        "Anzahl in Gold": sum(y_true),
        "Anzahl Predicted": sum(y_pred)
    })

# Ausgabe
df_metrics = pd.DataFrame(rows).sort_values("Label")
print("\n📊 Metriken pro Kategorie:")
print(df_metrics.to_string(index=False))

100%|█████████████████████████████████████| 200/200 [00:00<00:00, 111788.49it/s]


🔍 Gesamtergebnis 2. Durchlauf (alle Goldstandard-Dateien)
Precision: 0.276
Recall:    0.175
F1-Score:  0.214

📊 Metriken pro Kategorie:
Label  Precision  Recall  F1-Score  Anzahl in Gold  Anzahl Predicted
 DATE      0.503   0.324     0.394             290               187
EVENT      0.120   0.084     0.099             237               167
  LOC      0.239   0.107     0.148             403               180
 TIME      0.456   0.385     0.417             213               180
TOPIC      0.034   0.024     0.028             255               174


In [20]:
import json
from tqdm import tqdm
from collections import defaultdict
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd

# === Dateien laden ===
with open("../../data/data_annotated.json", encoding="utf-8") as f:
    gold_data = json.load(f)

with open("../../data/NER/gemma2/prompt2/gemma2_prompt2_2_as_goldstructure.json", encoding="utf-8") as f:
    gemma_data = json.load(f)

# === Zu vergleichende Labels ===
label_names = ["LOC", "DATE", "TIME", "EVENT", "TOPIC"]

# === Entitäten extrahieren ===
def extract_entities(entry):
    ent_dict = {}
    for ent in entry.get("entities", []):
        label = ent.get("label")
        text = ent.get("text", "").strip()
        if label and text:
            ent_dict.setdefault(label, []).append(text)
    return ent_dict

# === Gemeinsame Dateien filtern ===
gold_files = {entry["file_name"] for entry in gold_data}
gemma_files = {entry["file_name"] for entry in gemma_data}
common_files = gold_files & gemma_files

filtered_gold = [entry for entry in gold_data if entry["file_name"] in common_files]
gemma_dict = {entry["file_name"]: entry for entry in gemma_data if entry["file_name"] in common_files}

# === Vergleich vorbereiten ===
y_true_all = []
y_pred_all = []

# pro Label getrennte Zählung
y_true_per_label = defaultdict(list)
y_pred_per_label = defaultdict(list)

# === Vergleichsloop ===
for gold_entry in tqdm(filtered_gold):
    file_name = gold_entry["file_name"]
    gold_ents = extract_entities(gold_entry)
    pred_entry = gemma_dict.get(file_name)
    pred_ents = extract_entities(pred_entry) if pred_entry else {}

    for label in label_names:
        gold_texts = set(t.strip() for t in gold_ents.get(label, []) if isinstance(t, str))
        val = pred_ents.get(label)

        if isinstance(val, list):
            pred_texts = set(t.strip() for t in val if isinstance(t, str))
        elif isinstance(val, str):
            pred_texts = {val.strip()}
        else:
            pred_texts = set()

        # True Positives und False Negatives
        for text in gold_texts:
            if text in pred_texts:
                y_true_all.append(1)
                y_pred_all.append(1)
                y_true_per_label[label].append(1)
                y_pred_per_label[label].append(1)
                pred_texts.remove(text)
            else:
                y_true_all.append(1)
                y_pred_all.append(0)
                y_true_per_label[label].append(1)
                y_pred_per_label[label].append(0)

        # Verbleibende Vorhersagen = False Positives
        for text in pred_texts:
            y_true_all.append(0)
            y_pred_all.append(1)
            y_true_per_label[label].append(0)
            y_pred_per_label[label].append(1)

# === Gesamtmetriken ===
precision, recall, f1, _ = precision_recall_fscore_support(
    y_true_all, y_pred_all, average="binary", zero_division=0
)

print("\n🔍 Gesamtergebnis (nur gemeinsame Dateien)")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")
print(f"F1-Score:  {f1:.3f}")

# === Metriken pro Label ===
rows = []
for label in label_names:
    y_true = y_true_per_label[label]
    y_pred = y_pred_per_label[label]

    if not y_true and not y_pred:
        precision = recall = f1 = 0.0
    else:
        precision, recall, f1, _ = precision_recall_fscore_support(
            y_true, y_pred, average="binary", zero_division=0
        )

    rows.append({
        "Label": label,
        "Precision": round(precision, 3),
        "Recall": round(recall, 3),
        "F1-Score": round(f1, 3),
        "Anzahl in Gold": sum(y_true),
        "Anzahl Predicted": sum(y_pred)
    })

# Ausgabe als Tabelle
df_metrics = pd.DataFrame(rows).sort_values("Label")
print("\n📊 Metriken pro Kategorie:")
print(df_metrics.to_string(index=False))

100%|██████████████████████████████████████| 173/173 [00:00<00:00, 79215.57it/s]


🔍 Gesamtergebnis (nur gemeinsame Dateien)
Precision: 0.276
Recall:    0.205
F1-Score:  0.235

📊 Metriken pro Kategorie:
Label  Precision  Recall  F1-Score  Anzahl in Gold  Anzahl Predicted
 DATE      0.503   0.376     0.430             250               187
EVENT      0.120   0.099     0.108             202               167
  LOC      0.239   0.129     0.167             334               180
 TIME      0.456   0.446     0.451             184               180
TOPIC      0.034   0.027     0.030             225               174


---
### Vergleich - Übereinstimmungen bei fehlerhafter Ausgabe des Jsons?

In [1]:
import json

with open("../../data/NER/gemma2/prompt2/gemma2_prompt2_1_as_goldstructure.json", encoding="utf-8") as f:
    prompt1 = json.load(f)

with open("../../data/NER/gemma2/prompt2/gemma2_prompt2_2_as_goldstructure.json", encoding="utf-8") as f:
    prompt2 = json.load(f)

# Set der file_names extrahieren
file_names1 = {entry['file_name'] for entry in prompt1}
file_names2 = {entry['file_name'] for entry in prompt2}


gemeinsame = file_names1 & file_names2


print(f"Anzahl der Einträge durchlauf1: {len(prompt1)}")
print(f"Anzahl der Einträge durchlauf2: {len(prompt2)}")
print(f"Gemeinsame Einträge (file_name): {len(gemeinsame)}")


Anzahl der Einträge durchlauf1: 183
Anzahl der Einträge durchlauf2: 173
Gemeinsame Einträge (file_name): 159


In [ ]:
---> es wurden bei unterschiedlichen Textfiles keine Entitäten extrahiert